In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [36]:
validation_data = pd.read_csv('/kaggle/input/trees-clf/hw_3_markup_data.txt', sep='\t')
train_data = pd.read_csv('/kaggle/input/trees-clf/train_data_hw5.tsv', sep='\t')

In [37]:
validation_dict = {}
for _, row in validation_data.iterrows():
    url = row['downloadUrl']
    label = row['is_conifer']
    validation_dict[url] = label

In [38]:
train_data['true_label'] = train_data['INPUT:image'].map(validation_dict)
golden_data = train_data[train_data['GOLDEN:result'].notna()].copy()

In [39]:
golden_data

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,INPUT:image,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,true_label
0,https://new-projects-team-public.s3.yandex.net...,True,False,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e3b2,0000548ef5--693819e66cad6f0284ef1add,24736b56d590c5d9d24c5a86485c26fa,APPROVED,2025-12-09T12:45:26.309,2025-12-09T12:45:39.510,False
10,https://new-projects-team-public.s3.yandex.net...,True,True,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e3bc,0000548ef5--693819fca0effc61c250e4cb,24736b56d590c5d9d24c5a86485c26fa,APPROVED,2025-12-09T12:45:48.258,2025-12-09T12:45:58.912,True
24,https://new-projects-team-public.s3.yandex.net...,False,False,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e3c0,0000548ef5--693819fdf30df95b3e77a075,6117719f6dd9341bf183a6e4dd28dcf2,APPROVED,2025-12-09T12:45:49.074,2025-12-09T12:46:04.796,False
35,https://new-projects-team-public.s3.yandex.net...,True,True,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e371,0000548ef5--69381a0246bc102b0d9a5827,cb8718798c8a927a105206f8c2ea7c85,APPROVED,2025-12-09T12:45:54.648,2025-12-09T12:46:19.860,True
49,https://new-projects-team-public.s3.yandex.net...,False,False,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e321,0000548ef5--69381a08ea870c753b655824,24736b56d590c5d9d24c5a86485c26fa,APPROVED,2025-12-09T12:46:00.037,2025-12-09T12:46:11.441,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,https://new-projects-team-public.s3.yandex.net...,False,False,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e394,0000548ef5--6939336597d217215d599c1e,efe59f6f9688752460754155ffde8822,APPROVED,2025-12-10T08:46:29.844,2025-12-10T08:46:51.115,False
4978,https://new-projects-team-public.s3.yandex.net...,False,False,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e364,0000548ef5--6939337caf18252a201b3b40,efe59f6f9688752460754155ffde8822,APPROVED,2025-12-10T08:46:52.897,2025-12-10T08:47:18.620,False
4985,https://new-projects-team-public.s3.yandex.net...,True,True,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e372,0000548ef5--69393388ba6371491518d023,ab52cfe87f1ca99d90a858762e8413d7,APPROVED,2025-12-10T08:47:04.081,2025-12-10T08:47:22.584,True
4990,https://new-projects-team-public.s3.yandex.net...,True,True,NaN,NaN,https://tasks.yandex.ru/task/5541621/0000548ef...,0000548ef5--693809fa1c0b1e506da2e3b4,0000548ef5--69393397e5e1e70ed1b8c7c4,efe59f6f9688752460754155ffde8822,APPROVED,2025-12-10T08:47:19.717,2025-12-10T08:47:28.979,True


In [41]:
golden_data['worker_answer'] = golden_data['OUTPUT:result']
golden_data['golden_answer'] = golden_data['GOLDEN:result']

In [42]:
golden_data['golden_answer'].unique()

array([False, True], dtype=object)

In [43]:
golden_data['worker_answer'].unique()

array([ True, False])

In [44]:
answers_df = golden_data[['ASSIGNMENT:task_id', 'ASSIGNMENT:worker_id', 'worker_answer']].copy()
answers_df.columns = ['task_id', 'worker_id', 'answer']

In [45]:
worker_performance = []
for worker_id in answers_df['worker_id'].unique():
    worker_data = golden_data[golden_data['ASSIGNMENT:worker_id'] == worker_id]
    if len(worker_data) > 0:
        accuracy = (worker_data['worker_answer'] == worker_data['golden_answer']).mean()
        worker_performance.append({
            'worker_id': worker_id,
            'skill_value': accuracy
        })

workers_skills = pd.DataFrame(worker_performance)

In [46]:
task_to_golden = golden_data.groupby('ASSIGNMENT:task_id')['golden_answer'].first().to_dict()

In [14]:
def majority_vote(answers: pd.DataFrame) -> pd.DataFrame:
    result = answers.groupby('task_id')['answer'].apply(lambda x: x.sum() > len(x) / 2).reset_index()
    result.columns = ['task_id', 'agg_answer']
    return result

In [15]:
def david_skene(answers: pd.DataFrame, max_iter=50) -> pd.DataFrame:
    tasks = answers['task_id'].unique()
    workers = answers['worker_id'].unique()
    
    task_to_idx = {task: idx for idx, task in enumerate(tasks)}
    worker_to_idx = {worker: idx for idx, worker in enumerate(workers)}
    
    n_tasks = len(tasks)
    n_workers = len(workers)
    
    response_matrix = np.full((n_tasks, n_workers), -1, dtype=np.int8)
    
    for _, row in answers.iterrows():
        task_idx = task_to_idx[row['task_id']]
        worker_idx = worker_to_idx[row['worker_id']]
        response_matrix[task_idx, worker_idx] = 1 if row['answer'] else 0
    
    p = np.zeros(n_tasks)
    for i in range(n_tasks):
        valid = response_matrix[i, :][response_matrix[i, :] >= 0]
        if len(valid) > 0:
            p[i] = valid.mean()
    
    confusion_matrices = np.zeros((n_workers, 2, 2))
    for j in range(n_workers):
        confusion_matrices[j] = np.array([[0.8, 0.2], [0.2, 0.8]])
    
    for iteration in range(max_iter):
        for i in range(n_tasks):
            prob_true = p[i] if p[i] > 0 else 0.5
            prob_false = 1 - prob_true
            
            log_prob_true = np.log(prob_true + 1e-10)
            log_prob_false = np.log(prob_false + 1e-10)
            
            for j in range(n_workers):
                if response_matrix[i, j] == -1:
                    continue
                
                r = response_matrix[i, j]
                log_prob_true += np.log(confusion_matrices[j, 1, r] + 1e-10)
                log_prob_false += np.log(confusion_matrices[j, 0, r] + 1e-10)
            
            max_log = max(log_prob_true, log_prob_false)
            prob_true_unnorm = np.exp(log_prob_true - max_log)
            prob_false_unnorm = np.exp(log_prob_false - max_log)
            
            p[i] = prob_true_unnorm / (prob_true_unnorm + prob_false_unnorm)
        
        for j in range(n_workers):
            confusion_matrices[j] = np.ones((2, 2)) * 0.1
            
            for i in range(n_tasks):
                if response_matrix[i, j] == -1:
                    continue
                
                r = response_matrix[i, j]
                confusion_matrices[j, 1, r] += p[i]
                confusion_matrices[j, 0, r] += (1 - p[i])
            
            confusion_matrices[j, 0, :] /= confusion_matrices[j, 0, :].sum()
            confusion_matrices[j, 1, :] /= confusion_matrices[j, 1, :].sum()
    
    result = pd.DataFrame({
        'task_id': tasks,
        'agg_answer': p > 0.5
    })
    
    return result

In [17]:
# алгоритм из ДЗ 4
def aggregation(answers: pd.DataFrame, workers_skills: pd.DataFrame) -> pd.DataFrame:
    
    answers = answers.merge(workers_skills, on='worker_id', how='left')
    answers['skill_value'] = answers['skill_value'].fillna(answers['skill_value'].median())
    
    tasks = answers['task_id'].unique()
    workers = answers['worker_id'].unique()
    n_tasks = len(tasks)
    n_workers = len(workers)
    
    task_to_idx = {task: idx for idx, task in enumerate(tasks)}
    worker_to_idx = {worker: idx for idx, worker in enumerate(workers)}
    
    response_matrix = np.full((n_tasks, n_workers), -1, dtype=np.int8)
    skill_vector = np.zeros(n_workers)
    
    for _, row in answers.iterrows():
        task_idx = task_to_idx[row['task_id']]
        worker_idx = worker_to_idx[row['worker_id']]
        response_matrix[task_idx, worker_idx] = 1 if row['answer'] else 0
        skill_vector[worker_idx] = row['skill_value']
    
    skill_min, skill_max = skill_vector.min(), skill_vector.max()
    if skill_max > skill_min:
        skill_norm = (skill_vector - skill_min) / (skill_max - skill_min)
    else:
        skill_norm = np.ones(n_workers) * 0.5
    
    mu = np.full(n_tasks, 0.5)
    
    alpha = np.full(n_workers, 0.8)
    beta_param = np.full(n_workers, 0.8)
    
    for j in range(n_workers):
        base_skill = 0.5 + 0.45 * skill_norm[j]
        alpha[j] = base_skill
        beta_param[j] = base_skill
    
    task_difficulty = np.zeros(n_tasks)
    
    task_counts = np.zeros(n_tasks)
    for i in range(n_tasks):
        valid_responses = response_matrix[i, :][response_matrix[i, :] >= 0]
        if len(valid_responses) > 0:
            mu[i] = valid_responses.mean()
            task_counts[i] = len(valid_responses)
    
    max_iterations = 100
    tolerance = 1e-7
    # фича с скорингом челов
    for iteration in range(max_iterations):
        old_mu = mu.copy()
        old_alpha = alpha.copy()
        old_beta = beta_param.copy()
        
        for i in range(n_tasks):
            prior_alpha = 2.0
            prior_beta = 2.0
            
            log_prob_y1 = 0
            log_prob_y0 = 0
            
            for j in range(n_workers):
                if response_matrix[i, j] == -1:
                    continue
                
                r_ij = response_matrix[i, j]
                
                if r_ij == 1:
                    prob_r_given_y1 = alpha[j]
                    prob_r_given_y0 = 1 - beta_param[j]
                else:
                    prob_r_given_y1 = 1 - alpha[j]
                    prob_r_given_y0 = beta_param[j]
                
                log_prob_y1 += np.log(prob_r_given_y1 + 1e-10)
                log_prob_y0 += np.log(prob_r_given_y0 + 1e-10)
            
            log_prior_y1 = (prior_alpha - 1) * np.log(0.5 + 1e-10)
            log_prior_y0 = (prior_beta - 1) * np.log(0.5 + 1e-10)
            
            log_prob_y1 += log_prior_y1
            log_prob_y0 += log_prior_y0
            
            max_log = max(log_prob_y1, log_prob_y0)
            prob_y1 = np.exp(log_prob_y1 - max_log)
            prob_y0 = np.exp(log_prob_y0 - max_log)
            
            mu[i] = prob_y1 / (prob_y1 + prob_y0)
            
            n_responses = np.sum(response_matrix[i, :] >= 0)
            if n_responses < 3:
                mu[i] = 0.7 * mu[i] + 0.3 * 0.5
        
        for j in range(n_workers):
            sum_mu = 0
            sum_1_minus_mu = 0
            sum_r_and_mu = 0
            sum_not_r_and_not_mu = 0
            count = 0
            
            for i in range(n_tasks):
                if response_matrix[i, j] == -1:
                    continue
                
                r_ij = response_matrix[i, j]
                mu_i = mu[i]
                
                sum_mu += mu_i
                sum_1_minus_mu += (1 - mu_i)
                
                if r_ij == 1:
                    sum_r_and_mu += mu_i
                else:
                    sum_not_r_and_not_mu += (1 - mu_i)
                
                count += 1
            
            if count > 0:
                smooth = 1.0
                
                alpha_new = (sum_r_and_mu + smooth) / (sum_mu + 2 * smooth)
                
                beta_new = (sum_not_r_and_not_mu + smooth) / (sum_1_minus_mu + 2 * smooth)
                
                skill_weight = 0.3
                alpha[j] = (1 - skill_weight) * alpha_new + skill_weight * (0.5 + 0.45 * skill_norm[j])
                beta_param[j] = (1 - skill_weight) * beta_new + skill_weight * (0.5 + 0.45 * skill_norm[j])
                
                alpha[j] = np.clip(alpha[j], 0.51, 0.99)
                beta_param[j] = np.clip(beta_param[j], 0.51, 0.99)
                
                accuracy = (sum_r_and_mu + sum_not_r_and_not_mu) / count
                if accuracy < 0.55:
                    alpha[j] = 0.51
                    beta_param[j] = 0.51
        
        for i in range(n_tasks):
            responses = response_matrix[i, response_matrix[i, :] >= 0]
            if len(responses) > 1:
                variance = np.var(responses)
                task_difficulty[i] = variance
        
        mu_change = np.max(np.abs(mu - old_mu))
        alpha_change = np.max(np.abs(alpha - old_alpha))
        beta_change = np.max(np.abs(beta_param - old_beta))
        
        if mu_change < tolerance and alpha_change < tolerance and beta_change < tolerance:
            break
    
    final_labels = np.zeros(n_tasks, dtype=bool)
    
    for i in range(n_tasks):
        n_responses = np.sum(response_matrix[i, :] >= 0)
        
        if n_responses >= 5:
            threshold = 0.5
        elif n_responses >= 3:
            threshold = 0.55
        else:
            threshold = 0.6
        
        if task_difficulty[i] > 0.2:
            threshold += 0.05
        
        final_labels[i] = mu[i] > threshold
    
    aggregated_result = pd.DataFrame({
        'task_id': tasks,
        'agg_answer': final_labels
    })
    
    aggregated_result = aggregated_result.sort_values('task_id').reset_index(drop=True)
    
    return aggregated_result[["task_id", "agg_answer"]]

In [47]:
mv_result = majority_vote(answers_df)
ds_result = david_skene(answers_df)
custom_result = aggregation(answers_df, workers_skills)

In [48]:
mv_result['golden'] = mv_result['task_id'].map(task_to_golden)
ds_result['golden'] = ds_result['task_id'].map(task_to_golden)
custom_result['golden'] = custom_result['task_id'].map(task_to_golden)

In [49]:
def evaluate_method(result_df, method_name):
    y_true = result_df['golden'].values
    y_pred = result_df['agg_answer'].values
    
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    
    print(f"\n{method_name}:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    
    return {'method': method_name, 'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec}

In [50]:
results = []
results.append(evaluate_method(mv_result, "Majority Vote"))
results.append(evaluate_method(ds_result, "David-Skene"))
results.append(evaluate_method(custom_result, "Custom Algorithm"))

results_df = pd.DataFrame(results)
results_df.head()


Majority Vote:
Accuracy: 0.7037
F1-score: 0.6308
Precision: 0.8200
Recall: 0.5125

David-Skene:
Accuracy: 0.7284
F1-score: 0.6986
Precision: 0.7727
Recall: 0.6375

Custom Algorithm:
Accuracy: 0.7222
F1-score: 0.6667
Precision: 0.8182
Recall: 0.5625


,method,accuracy,f1,precision,recall
0,Majority Vote,0.703704,0.630769,0.820000,0.5125
1,David-Skene,0.728395,0.698630,0.772727,0.6375
2,Custom Algorithm,0.722222,0.666667,0.818182,0.5625


Comment: нам важен recall, поэтому будем использовать аггрегацию через David-Scene

In [51]:
ds_result

,task_id,agg_answer,golden
0,0000548ef5--693809fa1c0b1e506da2e3b2,True,False
1,0000548ef5--693809fa1c0b1e506da2e3bc,True,True
2,0000548ef5--693809fa1c0b1e506da2e3c0,False,False
3,0000548ef5--693809fa1c0b1e506da2e371,True,True
4,0000548ef5--693809fa1c0b1e506da2e321,False,False
...,...,...,...
157,0000548ef5--693809fa1c0b1e506da2e36a,True,True
158,0000548ef5--693809fa1c0b1e506da2e373,True,True
159,0000548ef5--693809fa1c0b1e506da2e393,True,True
160,0000548ef5--693809fa1c0b1e506da2e362,True,False


In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

In [61]:
train_clean = train_data[train_data['GOLDEN:result'].isna()].copy()

In [62]:
train_clean['worker_answer'] = train_clean['OUTPUT:result']
train_answers = train_clean[['ASSIGNMENT:task_id', 'ASSIGNMENT:worker_id', 'worker_answer']].copy()
train_answers.columns = ['task_id', 'worker_id', 'answer']

In [63]:
aggregated_labels = david_skene(train_answers)

In [64]:
task_to_url = train_clean.groupby('ASSIGNMENT:task_id')['INPUT:image'].first().to_dict()
aggregated_labels['image_url'] = aggregated_labels['task_id'].map(task_to_url)

In [65]:
aggregated_labels.head()

,task_id,agg_answer,image_url
0,0000548ef5--693809b6d4e59077163241d0,True,https://new-projects-team-public.s3.yandex.net...
1,0000548ef5--6938098ed4e5907716323719,False,https://new-projects-team-public.s3.yandex.net...
2,0000548ef5--69380966d4e5907716322bbb,False,https://new-projects-team-public.s3.yandex.net...
3,0000548ef5--6938098ed4e59077163236a8,False,https://new-projects-team-public.s3.yandex.net...
4,0000548ef5--693809b6d4e59077163240c3,False,https://new-projects-team-public.s3.yandex.net...


In [67]:
aggregated_labels.agg_answer.value_counts()

agg_answer
False    1020
True      480
Name: count, dtype: int64

In [68]:
train_dataset = aggregated_labels[['image_url', 'agg_answer']].copy()
train_dataset.columns = ['url', 'label']
train_dataset['label'] = train_dataset['label'].astype(int)

In [72]:
validation_data.dropna(inplace = True)
validation_data

,is_conifer,downloadUrl
0,True,https://new-projects-team-public.s3.yandex.net...
1,False,https://new-projects-team-public.s3.yandex.net...
2,True,https://new-projects-team-public.s3.yandex.net...
3,False,https://new-projects-team-public.s3.yandex.net...
4,False,https://new-projects-team-public.s3.yandex.net...
...,...,...
195,True,https://new-projects-team-public.s3.yandex.net...
196,False,https://new-projects-team-public.s3.yandex.net...
197,True,https://new-projects-team-public.s3.yandex.net...
198,True,https://new-projects-team-public.s3.yandex.net...


In [73]:
val_dataset = validation_data[['downloadUrl', 'is_conifer']].copy()
val_dataset.columns = ['url', 'label']
val_dataset['label'] = val_dataset['label'].astype(int)

In [86]:
def download_image(url):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        return img
    except:
        return None

In [87]:
print("Downloading training images...")
train_images = []
train_labels = []

for idx, row in tqdm(train_dataset.iterrows(), total=len(train_dataset)):
    img = download_image(row['url'])
    if img is not None:
        train_images.append(img)
        train_labels.append(row['label'])

100%|██████████| 1500/1500 [12:21<00:00,  2.02it/s]


In [88]:
print("Downloading validation images...")
val_images = []
val_labels = []

for idx, row in tqdm(val_dataset.iterrows(), total=len(val_dataset)):
    img = download_image(row['url'])
    if img is not None:
        val_images.append(img)
        val_labels.append(row['label'])

100%|██████████| 200/200 [01:43<00:00,  1.94it/s]


In [89]:
from PIL import ImageEnhance

def augment_images(images, labels):
    augmented_images = []
    augmented_labels = []
    
    for img, label in tqdm(zip(images, labels), total=len(images), desc="Augmenting"):
        for rotation in [0, 90, 180, 270]:
            for brightness in [0.7, 1.0, 1.3]:
                img_aug = img.rotate(rotation, expand=True)
                
                enhancer = ImageEnhance.Brightness(img_aug)
                img_aug = enhancer.enhance(brightness)
                
                augmented_images.append(img_aug)
                augmented_labels.append(label)
    
    return augmented_images, augmented_labels

In [90]:
train_images_aug, train_labels_aug = augment_images(train_images, train_labels)
val_images_aug, val_labels_aug = augment_images(val_images, val_labels)

Augmenting: 100%|██████████| 200/200 [00:00<00:00, 771.66it/s]


In [91]:
indices = np.random.permutation(len(train_images_aug))
train_images_aug = [train_images_aug[i] for i in indices]
train_labels_aug = [train_labels_aug[i] for i in indices]

indices = np.random.permutation(len(val_images_aug))
val_images_aug = [val_images_aug[i] for i in indices]
val_labels_aug = [val_labels_aug[i] for i in indices]

print(f"Training samples: {len(train_images_aug)}")
print(f"Validation samples: {len(val_images_aug)}")

Training samples: 18000
Validation samples: 2400


In [92]:
class PreloadedDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

In [93]:
train_transform = transforms.Compose([
    transforms.Resize(320),
    transforms.RandomCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(320),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [94]:
train_ds = PreloadedDataset(train_images_aug, train_labels_aug, transform=train_transform)
val_ds = PreloadedDataset(val_images_aug, val_labels_aug, transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [96]:
model = models.inception_v3(pretrained=True)
model.aux_logits = False

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1)
)

model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [97]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [98]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images = images.to(device)
        labels = labels.float().to(device)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        predictions = (torch.sigmoid(outputs) > 0.5).long()
        correct += (predictions == labels.long()).sum().item()
        total += labels.size(0)
    
    return running_loss / len(loader), correct / total

In [99]:
def validate(model, loader, device):
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation"):
            images = images.to(device)
            labels = labels.float().to(device)
            
            outputs = model(images).squeeze()
            probs = torch.sigmoid(outputs)
            predictions = (probs > 0.5).long()
            
            all_preds.extend(predictions.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.long().cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_probs)
    cm = confusion_matrix(all_labels, all_preds)
    
    return acc, f1, precision, recall, roc_auc, cm

In [100]:
num_epochs = 20
best_val_acc = 0.0
best_roc_auc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_acc, val_f1, val_precision, val_recall, val_roc_auc, val_cm = validate(model, val_loader, device)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Acc: {val_acc:.4f}")
    print(f"Val F1: {val_f1:.4f}")
    print(f"Val Precision: {val_precision:.4f}")
    print(f"Val Recall: {val_recall:.4f}")
    print(f"Val ROC-AUC: {val_roc_auc:.4f}")
    print(f"Confusion Matrix:\n{val_cm}")
    
    scheduler.step(val_acc)
    
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_acc': val_acc,
        'val_f1': val_f1,
        'val_roc_auc': val_roc_auc
    }
    torch.save(checkpoint, f'checkpoint_epoch_{epoch+1}.pth')
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(checkpoint, 'best_model_acc.pth')
        print(f"Saved best accuracy model: {best_val_acc:.4f}")
    
    if val_roc_auc > best_roc_auc:
        best_roc_auc = val_roc_auc
        torch.save(checkpoint, 'best_model_roc_auc.pth')
        print(f"Saved best ROC-AUC model: {best_roc_auc:.4f}")


Epoch 1/20


Validation: 100%|██████████| 75/75 [00:10<00:00,  7.03it/s]


Train Loss: 0.5855, Train Acc: 0.6992
Val Acc: 0.6533
Val F1: 0.5331
Val Precision: 0.8162
Val Recall: 0.3958
Val ROC-AUC: 0.8092
Confusion Matrix:
[[1093  107]
 [ 725  475]]
Saved best accuracy model: 0.6533
Saved best ROC-AUC model: 0.8092

Epoch 2/20


Validation: 100%|██████████| 75/75 [00:10<00:00,  7.01it/s]


Train Loss: 0.5614, Train Acc: 0.7208
Val Acc: 0.5833
Val F1: 0.3395
Val Precision: 0.8185
Val Recall: 0.2142
Val ROC-AUC: 0.8137
Confusion Matrix:
[[1143   57]
 [ 943  257]]
Saved best ROC-AUC model: 0.8137

Epoch 3/20


...

Train Loss: 0.5109, Train Acc: 0.7494
Val Acc: 0.6879
Val F1: 0.5954
Val Precision: 0.8464
Val Recall: 0.4592
Val ROC-AUC: 0.8321
Confusion Matrix:
[[1100  100]
 [ 649  551]]

Epoch 19/20


Validation: 100%|██████████| 75/75 [00:10<00:00,  6.98it/s]


Train Loss: 0.5078, Train Acc: 0.7503
Val Acc: 0.6850
Val F1: 0.5927
Val Precision: 0.8384
Val Recall: 0.4583
Val ROC-AUC: 0.8284
Confusion Matrix:
[[1094  106]
 [ 650  550]]

Epoch 20/20


Validation: 100%|██████████| 75/75 [00:10<00:00,  6.97it/s]


Train Loss: 0.5094, Train Acc: 0.7521
Val Acc: 0.6771
Val F1: 0.5822
Val Precision: 0.8244
Val Recall: 0.4500
Val ROC-AUC: 0.8216
Confusion Matrix:
[[1085  115]
 [ 660  540]]


In [101]:
print(f"\nBest Validation Accuracy: {best_val_acc:.4f}")
print(f"Best Validation ROC-AUC: {best_roc_auc:.4f}")


Best Validation Accuracy: 0.7058
Best Validation ROC-AUC: 0.8387


In [103]:
checkpoint = torch.load('checkpoints/best_model_roc_auc.pth', weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])

val_acc, val_f1, val_precision, val_recall, val_roc_auc, val_cm = validate(model, val_loader, device)

print("\nFinal metrics on validation set (best ROC-AUC model):")
print(f"Accuracy: {val_acc:.4f}")
print(f"F1-score: {val_f1:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")
print(f"ROC-AUC: {val_roc_auc:.4f}")
print(f"Confusion Matrix:\n{val_cm}")

Validation: 100%|██████████| 75/75 [00:10<00:00,  6.96it/s]



Final metrics on validation set (best ROC-AUC model):
Accuracy: 0.6512
F1-score: 0.5148
Precision: 0.8457
Recall: 0.3700
ROC-AUC: 0.8215
Confusion Matrix:
[[1119   81]
 [ 756  444]]
